Objectives:
   1. Open catchments dataframe (polygons over which landscape data will be collected)
   2. Collect catchment statistics from Earth Engine datasets
   3. Export to a table that can be used in RandomForest

## 1. Import packages / set working directory

In [1]:
import os
import io

import earthpy as et
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import geemap
from shapely.geometry import shape
import ee
#import elevation
import math

# Initialize Google Earth Engine to generate area of interest 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

%matplotlib inline

In [2]:
# Set working directory

# if the desired path exists:
data_dir = os.path.join(et.io.HOME, 'Dropbox',
                        'cu_earthdata_certificate_2021', 'earthlab_project', 'data')
if os.path.exists(data_dir):
    # set working directory:
    os.chdir(data_dir)
    print("path exists")
else:
    print("path does not exist, making new path")
    os.makedirs(data_dir)
    os.chdir(data_dir)

path exists


## 2. Defining the bounds
Let's import the bounds of our Area of Study

In [3]:
lis = ['Colorado','Utah', 'Wyoming', 'New Mexico', 'Arizona']
states = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.inList('NAME', lis))

states_gdf = geemap.ee_to_geopandas(states, selectors = ['NAME'])
print("The crs of your area of interest df is", states_gdf.crs)
states_gdf.head()

bounding_box = states_gdf.envelope
states_bb = gpd.GeoDataFrame(gpd.GeoSeries(bounding_box), columns=['geometry'])
states_bb

An error occurred while downloading. 
 Retrying ...
Generating URL ...
Please wait ...
The crs of your area of interest df is epsg:4326


,geometry
0,"POLYGON ((-109.05041 31.33222, -103.00203 31.3..."
1,"POLYGON ((-114.05288 36.99765, -109.04157 36.9..."
2,"POLYGON ((-109.06020 36.99245, -102.04153 36.9..."
3,"POLYGON ((-111.05454 40.99478, -104.05226 40.9..."
4,"POLYGON ((-114.81657 31.33217, -109.04516 31.3..."


In [4]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [5]:
elevation = ee.Image('USGS/NED').select('elevation')

terrain = ee.Image("CGIAR/SRTM90_V4").float()                                    

vis_params={
    "min": 0, 
    "max": 4000,
    "palette": ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}
Map.addLayer(terrain, vis_params, 'terrain')

In [6]:
# Import file containing full catchment for each burned lake
# from local drive generated in 01_download_clip_merge.ipynb
file_path = os.path.join('land_only_whole_ctchmt_aqst.shp')

# Geodataframe used in folium map
land_catchmts = gpd.read_file(file_path)

# EE object used for Earth Engine collections and calculations
#small_gdf = land_catchmts[["Hylak_id", "geometry"]]
land_ctchmts_ee = geemap.geopandas_to_ee(land_catchmts)

In [7]:
# Estimating the slope
slope = ee.Terrain.slope(terrain).float()

# Estimating the hillshade
aspect = ee.Terrain.aspect(terrain).float()
northness = aspect.cos().rename('northness')
eastness = aspect.sin().rename('eastness')

# Indices from Theobald and CSP
mtpi = ee.Image('CSP/ERGo/1_0/US/mTPI').rename('mtpi').resample('bicubic')
chili = ee.Image('CSP/ERGo/1_0/US/CHILI').rename('chili').resample('bicubic')
tdiv = ee.Image(
    'CSP/ERGo/1_0/US/topoDiversity').rename('tdiv').resample('bicubic')
physd = ee.Image(
    'CSP/ERGo/1_0/US/physioDiversity').rename('physd').resample('bicubic')
landform = ee.Image(
    'CSP/ERGo/1_0/US/landforms').rename('landform').resample().toFloat()

# Adding the slope and hillshade to the dem
newterrain = terrain.addBands(slope).addBands(aspect).addBands(
    chili).addBands(tdiv).addBands(physd).addBands(landform)

In [8]:
# Import area of interest

In [9]:
# Indices from Theobald and CSP
mtpi = ee.Image('CSP/ERGo/1_0/US/mTPI').rename('mtpi').resample('bicubic')
chili = ee.Image('CSP/ERGo/1_0/US/CHILI').rename('chili').resample('bicubic')
tdiv = ee.Image(
    'CSP/ERGo/1_0/US/topoDiversity').rename('tdiv').resample('bicubic')
physd = ee.Image(
    'CSP/ERGo/1_0/US/physioDiversity').rename('physd').resample('bicubic')
landform = ee.Image(
    'CSP/ERGo/1_0/US/landforms').rename('landform').resample().toFloat()

# add topographic bands to the image
topo = ee.Image.cat(elevation, aspect, northness, eastness,
                    slope, chili, tdiv, mtpi, physd, landform).clip(land_catchmts)

In [10]:
reduced_fc = newterrain.reduceRegions(
    reducer=ee.Reducer.mean(),
    collection=land_ctchmts_ee,
    scale=30)

In [11]:
# Function to Convert Feature Classes to Pandas Dataframe
# Adapted from: https://events.hpc.grnet.gr/event/47/material/1/12.py
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    # Features is a list of dict with the output
    features = fc.getInfo()['features']

    dictarr = []

    for f in features:
        # Store all attributes in a dict
        attr = f['properties']
        # and treat geometry separately
        attr['geometry'] = f['geometry']  # GeoJSON Feature!
        # attr['geometrytype'] = f['geometry']['type']
        dictarr.append(attr)

    df = gpd.GeoDataFrame(dictarr)
    # Convert GeoJSON features to shape
    df['geometry'] = map(lambda s: shape(s), df.geometry)
    return df

In [12]:
# Convert to Pandas Dataframe
df_image_red = fc2df(reduced_fc)

In [13]:
df_image_red.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 547 entries, 0 to 546
Data columns (total 49 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Hylak_id    547 non-null    int64  
 1   SiteID      547 non-null    object 
 2   TZID        547 non-null    object 
 3   aspect      547 non-null    float64
 4   blue        547 non-null    float64
 5   blue_sd     543 non-null    float64
 6   chili       547 non-null    int64  
 7   chl_a       88 non-null     float64
 8   clouds      547 non-null    float64
 9   date        547 non-null    object 
 10  date_only   547 non-null    int64  
 11  date_unity  547 non-null    object 
 12  date_utc    547 non-null    object 
 13  doc         34 non-null     float64
 14  elevation   547 non-null    int64  
 15  endtime     547 non-null    object 
 16  green       547 non-null    float64
 17  green_sd    543 non-null    float64
 18  id          547 non-null    object 
 19  landform    547 non-n

In [15]:
# Export file to local drive
out_path = os.path.join("lake_topography.csv")
df_image_red.to_csv(out_path)